# Import

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import os, random

import torch
import torch.nn as nn
from torch.nn.init import normal_
import torch.nn.functional as F

# 하이퍼파라미터 설정

In [2]:
# 하이퍼파라미터 
class cfg: 
    gpu_idx = 0
    device = torch.device("cuda:{}".format(gpu_idx) if torch.cuda.is_available() else "cpu")
    top_k = 25
    seed = 42
    neg_ratio = 100

In [3]:
torch.cuda.is_available()

True

# 시드 고정

In [4]:
# 시드 고정 
def seed_everything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    
seed_everything(cfg.seed)

# Data Load

In [5]:
# 경로 설정
data_path = '../data'
saved_path = './saved'
output_path = './submission'

### 데이터 불러오기
- history_data : 시청 시작 데이터
- profile_data : 프로필 정보 
- meta_data : 콘텐츠 일반 메타 정보

In [6]:
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')

## 중복 데이터 제거 ##
history_df = history_df[['profile_id', 'log_time', 'album_id']].drop_duplicates(subset=['profile_id', 'album_id', 'log_time']).sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)
history_df['rating']=1
history_df.head(3)

profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')

<ipython-input-6-aba586203ad9>:9: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')


# Train / Valid split

In [7]:
## user 별 전체 데이터중 80% train / 20% valid 사용 ## 

In [8]:
### user 별 전체 데이터*0.8 에 해당해는 데이터 개수 행 추가 ###
count_df = history_df.groupby(['profile_id']).count()
count_df = count_df[['album_id']]
count_df['train_count'] = count_df[['album_id']].apply(lambda x : x*(0.8))
count_df

,album_id,train_count
profile_id,,
3,21,16.8
5,543,434.4
7,2,1.6
12,7,5.6
16,3,2.4
...,...,...
33022,2,1.6
33023,12,9.6
33026,1,0.8


In [9]:
### 정확한 valid 분리를 위하여 각 user 별 시간순으로 정렬 ###
history_df = history_df.sort_values(['log_time'])
history_df

,profile_id,log_time,album_id,rating
798337,25844,20220301000418,18024,1
798338,25844,20220301000531,1881,1
185888,4783,20220301000656,201,1
798339,25844,20220301000668,4608,1
101611,2794,20220301000805,2641,1
...,...,...,...,...
250052,6435,20220430235415,2467,1
64322,2086,20220430235656,2184,1
313534,8440,20220430235710,348,1
292774,7703,20220430235855,188,1


In [10]:
### user별 시간순으로 정렬된 history_df를 이용하여 먼저본 80% 의 데이터를 train에 넣어주고 나중에 본 20%를 valid 에 넣어주기 ###
history_df = history_df.sort_values(['log_time'])

count_dict = defaultdict(int)

history_matrix = history_df.values
train_data=[]
valid_data=[]
for row in tqdm(history_matrix):
    profile_id = row[0]
    if count_dict[profile_id]<count_df.loc[profile_id,'train_count']:
        count_dict[profile_id]+=1
        train_data.append(row)
    else:
        valid_data.append(row)

  0%|          | 0/899252 [00:00<?, ?it/s]

In [11]:
### train_data 를 DataFrame 변환 ###
train_data_df = pd.DataFrame(train_data,columns=history_df.columns)
train_data_df

,profile_id,log_time,album_id,rating
0,25844,20220301000418,18024,1
1,25844,20220301000531,1881,1
2,4783,20220301000656,201,1
3,25844,20220301000668,4608,1
4,2794,20220301000805,2641,1
...,...,...,...,...
722707,7562,20220430233135,114,1
722708,18994,20220430233509,4053,1
722709,18994,20220430233524,818,1
722710,18994,20220430233542,818,1


In [12]:
### valid_data 를 DataFrame 변환 ###
valid_data_df = pd.DataFrame(valid_data,columns=history_df.columns)
valid_data_df

,profile_id,log_time,album_id,rating
0,6967,20220301101930,1465,1
1,6967,20220301102276,1747,1
2,6967,20220301102572,6529,1
3,6967,20220301102663,6530,1
4,6967,20220301102871,6531,1
...,...,...,...,...
176535,5597,20220430235403,2519,1
176536,6435,20220430235415,2467,1
176537,2086,20220430235656,2184,1
176538,8440,20220430235710,348,1


# 전체 데이터를 통하여 table 생성

In [13]:
## Train 과 Valid로 나눈 데이터중 Train 데이터를 이용하여 pivot_table을 생성하면 ##
## tarin에는 없고 valid에만 있는 (profile_id - album_id)가 존재하여 ##
## 행렬 크기가 달라지므로 전체 데이터를 이용하여 pivot_table 생성 ##

### 예시 ###
train_n_users = train_data_df.profile_id.nunique()
train_n_items = train_data_df.album_id.nunique()
print(train_n_users,train_n_items)

n_users = history_df.profile_id.nunique()
n_items = history_df.album_id.nunique()
print(n_users,n_items)
#############

8311 19657
8311 20695


In [14]:
## (8311,19657) 과 (8311,20695)로 차이 발생 --> 정확한 평가 어려움 ##
## 따라서 전체 데이터를 이용하여 table 생성

In [15]:
n_users = history_df.profile_id.nunique()
n_items = history_df.album_id.nunique()
print(n_users,n_items)

8311 20695


In [16]:
## 데이터가 있는 행과 열을 기준으로 table 형성을 위하여 rating 기준으로 dataframe 생성 ##

In [17]:
ratings_matrix_df = history_df.pivot_table('rating',index='profile_id',columns='album_id')
ratings_matrix_df.head()

album_id,0,1,2,3,4,5,6,7,8,9,...,25877,25893,25894,25895,25898,25912,25913,25914,25915,25916
profile_id,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
## ratings_matrix_df 와 동일한 크기의 zero DataFrame 생성 ##

In [19]:
ratings_total_matrix_df = pd.DataFrame(np.zeros(ratings_matrix_df.values.shape),index=ratings_matrix_df.index,columns=ratings_matrix_df.columns)
ratings_total_matrix_df.head()

album_id,0,1,2,3,4,5,6,7,8,9,...,25877,25893,25894,25895,25898,25912,25913,25914,25915,25916
profile_id,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
## zero DataFrame을 train 데이터만을 이용하여 (profile_id , album_id) 쌍의 개수를 세주기 ##

In [21]:
train_data = train_data_df.values

for row in tqdm(range(train_data.shape[0])): 
    row_data =train_data[row] # row_data = profile_id ,log_time, album_id, rating
    ratings_total_matrix_df.loc[row_data[0],row_data[2]]+=1

ratings_total_matrix_df.head()

  0%|          | 0/722712 [00:00<?, ?it/s]

album_id,0,1,2,3,4,5,6,7,8,9,...,25877,25893,25894,25895,25898,25912,25913,25914,25915,25916
profile_id,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 유저별로 최대 1 최소 0 으로 Minmaxscaler 적용

In [22]:
## MinMaxScaler 적용을 위해 transpose() 후 다시 transpose() 적용 ##
scaler = MinMaxScaler()
ratings_minmax_matrix_t = scaler.fit_transform(ratings_total_matrix_df.transpose())
ratings_minmax_matrix_df_t = pd.DataFrame(ratings_minmax_matrix_t,index=ratings_total_matrix_df.columns,columns=ratings_total_matrix_df.index)
ratings_minmax_matrix_df = ratings_minmax_matrix_df_t.transpose()
ratings_minmax_matrix_df.head()

album_id,0,1,2,3,4,5,6,7,8,9,...,25877,25893,25894,25895,25898,25912,25913,25914,25915,25916
profile_id,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# (profile_id와 index) 와 (album_id와 columns) 변환을 위한 dictionary 생성

In [23]:
index_list = list(ratings_minmax_matrix_df.index)
column_list = list(ratings_minmax_matrix_df.columns)

print('index 의 전체 길이 :',len(index_list),'columns 의 전체 길이 :',len(column_list))

real_to_fake_user = {real:fake for fake,real in enumerate(index_list)} # profile_id -> index
fake_to_real_user = {fake:real for fake,real in enumerate(index_list)} # index -> profile_id

real_to_fake_album = {real:fake for fake,real in enumerate(column_list)} # album_id -> column
fake_to_real_album = {fake:real for fake,real in enumerate(column_list)} # column -> album_id

print('profile_id 33032의 index 번호 :',real_to_fake_user[33032] ,'index 번호 8310의 profile_id :',fake_to_real_user[8310])
print('album_id 25916의 column 번호 :',real_to_fake_album[25916] ,'column 번호 20694의 album_id :',fake_to_real_album[20694])

index 의 전체 길이 : 8311 columns 의 전체 길이 : 20695
profile_id 33032의 index 번호 : 8310 index 번호 8310의 profile_id : 33032
album_id 25916의 column 번호 : 20694 column 번호 20694의 album_id : 25916


# 필요한 정보 추출

In [24]:
# 유저 특징 정보 추출 
profile_df = profile_df.set_index('profile_id')
user_features = profile_df[['age']].to_dict()
print("user_id 3의 age 정보 :", user_features['age'][3]) # 실제 id

user_id 3의 age 정보 : 5


In [25]:
# 아이템 특징 정보 추출 
meta_df = meta_df.set_index('album_id')

# 범주형 데이터를 수치형 데이터로 변경 
le = LabelEncoder()
meta_df['genre_mid'] = le.fit_transform(meta_df['genre_mid'])
item_features = meta_df[['genre_mid']].to_dict()
print("album_id 749의 genre_mid 정보 :", item_features['genre_mid'][749]) # 실제 album_id

album_id 749의 genre_mid 정보 : 1


In [26]:
# 추출한 특징 정보의 속성을 저장 
cfg.n_genres = meta_df['genre_mid'].nunique()
cfg.n_continuous_feats = 1 

In [27]:
cfg.n_users = n_users
cfg.n_items = n_items

# NeuMF 구현
## baseline 모델에서 이진분류가 아닌 회귀문제로 생각하기 위하여 모델 마지막 output에 sigmoid 추가

![](http://drive.google.com/uc?export=view&id=1tpajTLipLoFdvLICO-alAxeoKAE8-k61)

In [28]:
class NeuMF(nn.Module):
    """Neural Matrix Factorization Model
        참고 문헌 : https://arxiv.org/abs/1708.05031

    예시 :
        model = NeuMF(cfg) 
        output = model.forward(user_ids, item_ids, [feat0, feat1]) 
    """
    def __init__(self, cfg):
        """ 
        Args:
            cfg : config 파일로 네트워크 생성에 필요한 정보들을 담고 있음 
        """
        super(NeuMF, self).__init__()
        self.n_users = cfg.n_users
        self.n_items = cfg.n_items
        self.emb_dim = cfg.emb_dim
        self.layer_dim = cfg.layer_dim
        self.n_continuous_feats = cfg.n_continuous_feats
        self.n_genres = cfg.n_genres
        self.dropout = cfg.dropout
        self.build_graph()

    def build_graph(self):
        """Neural Matrix Factorization Model 생성
            구현된 모습은 위의 그림을 참고 
        """
        self.user_embedding_mf = nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.emb_dim)
        self.item_embedding_mf = nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.emb_dim)
        
        self.user_embedding_mlp = nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.emb_dim)
        self.item_embedding_mlp = nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.emb_dim)
                
        self.genre_embeddig = nn.Embedding(num_embeddings=self.n_genres, embedding_dim=self.n_genres//2)
        
        self.mlp_layers = nn.Sequential(
            nn.Linear(2*self.emb_dim + self.n_genres//2 + self.n_continuous_feats, self.layer_dim), 
            nn.ReLU(), 
            nn.Dropout(p=self.dropout), 
            nn.Linear(self.layer_dim, self.layer_dim//2), 
            nn.ReLU(), 
            nn.Dropout(p=self.dropout)
        )
        self.affine_output = nn.Linear(self.layer_dim//2 + self.emb_dim, 1)
        self.apply(self._init_weights)
        

    def _init_weights(self, module):
        if isinstance(module, nn.Embedding):
            normal_(module.weight.data, mean=0.0, std=0.01)
        elif isinstance(module, nn.Linear):
            normal_(module.weight.data, 0, 0.01)
            if module.bias is not None:
                module.bias.data.fill_(0.0)
    
    def forward(self, user_indices, item_indices, feats):
        """ 
        Args:
            user_indices : 유저의 인덱스 정보 
                ex) tensor([ 3100,  3100,  ..., 14195, 14195])
            item_indices : 아이템의 인덱스 정보
                ex) tensor([   50,    65,   ..., 14960, 11527])
            feats : 특징 정보 
        Returns: 
            output : 유저-아이템 쌍에 대한 추천 결과 
                ex) tensor([  9.4966,  22.0261, ..., -19.3535, -23.0212])
        """
        user_embedding_mf = self.user_embedding_mf(user_indices)
        item_embedding_mf = self.item_embedding_mf(item_indices)
        mf_output = torch.mul(user_embedding_mf, item_embedding_mf)
        
        user_embedding_mlp = self.user_embedding_mlp(user_indices)
        item_embedding_mlp = self.item_embedding_mlp(item_indices)
        genre_embedding_mlp = self.genre_embeddig(feats[1])
        input_feature = torch.cat((user_embedding_mlp, item_embedding_mlp, genre_embedding_mlp, feats[0].unsqueeze(1)), -1)
        mlp_output = self.mlp_layers(input_feature)
        
        output = torch.cat([mlp_output, mf_output], dim=-1)
        output = self.affine_output(output).squeeze(-1)
        return nn.Sigmoid()(output)

# 학습 및 추론에 필요한 데이터 셋 생성 코드 구현

In [29]:
def make_UIdataset(train, neg_ratio):
    """ 유저별 학습에 필요한 딕셔너리 데이터 생성 
    Args:
        train : 유저-아이템의 상호작용을 담은 행렬 
            ex) 
                array([[0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        ...,
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.]])
        neg_ratio : negative sampling 활용할 비율 
            ex) 3 (positive label 1개당 negative label 3개)
    Returns: 
        UIdataset : 유저별 학습에 필요한 정보를 담은 딕셔너리 
            ex) {'사용자 ID': [[positive 샘플, negative 샘플], ... , [1, 1, 1, ..., 0, 0]]}
                >>> UIdataset[3]
                    [array([   16,    17,    18, ...,  9586, 18991,  9442]),
                    array([5, 5, 5, ..., 5, 5, 5]),
                    array([4, 4, 4, ..., 5, 1, 1]),
                    array([1., 1., 1., ..., 0., 0., 0.])]
    """
    UIdataset = {}
    for user_id, items_by_user in enumerate(tqdm(train)):
        # 가짜 user id
        UIdataset[user_id] = []
        # positive 샘플 계산 
        pos_item_ids = np.where(items_by_user > 0)[0] # 가짜 아이템 id
        num_pos_samples = len(pos_item_ids)

        # negative 샘플 계산 (random negative sampling) 
        num_neg_samples = neg_ratio * num_pos_samples
        neg_items = np.where(items_by_user <= 0)[0] # 가짜 아이템 id
        neg_item_ids = np.random.choice(neg_items, min(num_neg_samples, len(neg_items)), replace=False)
        UIdataset[user_id].append(np.concatenate([pos_item_ids, neg_item_ids]))
        
        # feature 추출 
        features = []
        for item_id in np.concatenate([pos_item_ids, neg_item_ids]): 
            features.append(user_features['age'][fake_to_real_user[user_id]])
        UIdataset[user_id].append(np.array(features))
        
        features = []
        for item_id in np.concatenate([pos_item_ids, neg_item_ids]): 
            features.append(item_features['genre_mid'][fake_to_real_album[item_id]])
        UIdataset[user_id].append(np.array(features))
        
        # label 저장  
        pos_labels = items_by_user[pos_item_ids]
        neg_labels = items_by_user[neg_item_ids]
        UIdataset[user_id].append(np.concatenate([pos_labels, neg_labels]))

    return UIdataset

In [30]:
UIdataset = make_UIdataset(ratings_minmax_matrix_df.values, neg_ratio=cfg.neg_ratio)

  0%|          | 0/8311 [00:00<?, ?it/s]

In [31]:
# UIdataset [ 가짜 user _id ] = [[가짜 item id들],[나이정보],[장르정보],[label들 , label]]

In [32]:
def make_batchdata(user_indices, batch_idx, batch_size):
    """ 배치 데이터로 변환 
    Args:
        user_indices : 전체 유저의 인덱스 정보 
            ex) array([ 3100,  1800, 30098, ...,  2177, 11749, 20962])
        batch_idx : 배치 인덱스 (몇번째 배치인지)
            ex) 0 
        batch_size : 배치 크기 
            ex) 256 
    Returns 
        batch_user_ids : 배치내의 유저 인덱스 정보 
            ex) [22194, 22194, 22194, 22194, 22194, ...]
        batch_item_ids : 배치내의 아이템 인덱스 정보 
            ex) [36, 407, 612, 801, 1404, ...]
        batch_feat0 : 배치내의 유저-아이템 인덱스 정보에 해당하는 feature0 정보 
            ex) [6, 6, 6, 6, 6, ...]
        batch_feat1 : 배치내의 유저-아이템 인덱스 정보에 해당하는 feature1 정보 
            ex) [4,  4,  4, 23,  4, ...]
        batch_labels : 배치내의 유저-아이템 인덱스 정보에 해당하는 label 정보 
            ex) [1.0, 1.0, 1.0, 1.0, 1.0, ...]
    """
    batch_user_indices = user_indices[batch_idx*batch_size : (batch_idx+1)*batch_size]
    batch_user_ids = []
    batch_item_ids = []
    batch_feat0 = []
    batch_feat1 = []
    batch_labels = []
    for user_id in batch_user_indices:
        item_ids = UIdataset[user_id][0]
        feat0 = UIdataset[user_id][1]
        feat1 = UIdataset[user_id][2]
        labels = UIdataset[user_id][3]
        user_ids = np.full(len(item_ids), user_id)
        batch_user_ids.extend(user_ids.tolist())
        batch_item_ids.extend(item_ids.tolist())
        batch_feat0.extend(feat0.tolist())
        batch_feat1.extend(feat1.tolist())
        batch_labels.extend(labels.tolist())
    return batch_user_ids, batch_item_ids, batch_feat0, batch_feat1, batch_labels

def update_avg(curr_avg, val, idx):
    """ 현재 epoch 까지의 평균 값을 계산 
    """
    return (curr_avg * idx + val) / (idx + 1)

# 학습 및 검증 코드 생성

In [33]:
# cfg.n_users,cfg.n_items -> 8311 20695 개수 (0~8310 ,0~20694) 가짜 user id ,가짜 item id
def train_epoch(cfg, model, optimizer, criterion): 
    model.train()
    curr_loss_avg = 0.0

    user_indices = np.arange(cfg.n_users)
    np.random.RandomState(cfg.epoch).shuffle(user_indices)
    batch_num = int(len(user_indices) / cfg.batch_size) + 1 # 배치 전체 개수 int(8311/256) +1
    bar = tqdm(range(batch_num), leave=False)
    for step, batch_idx in enumerate(bar):
        user_ids, item_ids, feat0, feat1, labels = make_batchdata(user_indices, batch_idx, cfg.batch_size)
        # 배치 사용자 단위로 학습
        user_ids = torch.LongTensor(user_ids).to(cfg.device)
        item_ids = torch.LongTensor(item_ids).to(cfg.device)
        feat0 = torch.FloatTensor(feat0).to(cfg.device)
        feat1 = torch.LongTensor(feat1).to(cfg.device)
        labels = torch.FloatTensor(labels).to(cfg.device)
        labels = labels.view(-1, 1)

        # grad 초기화
        optimizer.zero_grad()

        # 모델 forward
        output = model.forward(user_ids, item_ids, [feat0, feat1])
        output = output.view(-1, 1)

        loss = criterion(output, labels)

        # 역전파
        loss.backward()

        # 최적화
        optimizer.step()    
        if torch.isnan(loss):
            print('Loss NAN. Train finish.')
            break
        curr_loss_avg = update_avg(curr_loss_avg, loss, step)
        
        msg = f"epoch: {cfg.epoch}, "
        msg += f"loss: {curr_loss_avg.item():.5f}, "
        msg += f"lr: {optimizer.param_groups[0]['lr']:.6f}"
        bar.set_description(msg)
    rets = {'losses': np.around(curr_loss_avg.item(), 5)}
    return rets

In [34]:
def recallk(actual, predicted, k = 25):
    """ label과 prediction 사이의 recall 평가 함수 
    Args:
        actual : 실제로 본 상품 리스트
        pred : 예측한 상품 리스트
        k : 상위 몇개의 데이터를 볼지 (ex : k=5 상위 5개의 상품만 봄)
    Returns: 
        recall_k : recall@k 
    """ 
    set_actual = set(actual)
    recall_k = len(set_actual & set(predicted[:k])) / min(k, len(set_actual))
    return recall_k

def unique(sequence):
    # preserves order
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def ndcgk(actual, predicted, k = 25):
    set_actual = set(actual)
    idcg = sum([1.0 / np.log(i + 2) for i in range(min(k, len(set_actual)))])
    dcg = 0.0
    unique_predicted = unique(predicted[:k])
    for i, r in enumerate(unique_predicted):
        if r in set_actual:
            dcg += 1.0 / np.log(i + 2)
    ndcg_k = dcg / idcg
    return ndcg_k

def evaluation(gt, pred):
    """ label과 prediction 사이의 recall, coverage, competition metric 평가 함수 
    Args:
        gt : 데이터 프레임 형태의 정답 데이터 
        pred : 데이터 프레임 형태의 예측 데이터 
    Returns: 
        rets : recall, ndcg, coverage, competition metric 결과 
            ex) {'recall': 0.123024, 'ndcg': 056809, 'coverage': 0.017455, 'score': 0.106470}
    """    
    gt = gt.groupby('profile_id')['album_id'].unique().to_frame().reset_index()
    gt.columns = ['profile_id', 'actual_list']

    evaluated_data = pd.merge(pred, gt, how = 'left', on = 'profile_id')

    evaluated_data['Recall@25'] = evaluated_data.apply(lambda x: recallk(x.actual_list, x.predicted_list), axis=1)
    evaluated_data['NDCG@25'] = evaluated_data.apply(lambda x: ndcgk(x.actual_list, x.predicted_list), axis=1)

    recall = evaluated_data['Recall@25'].mean()
    ndcg = evaluated_data['NDCG@25'] .mean()
    coverage = (evaluated_data['predicted_list'].apply(lambda x: x[:cfg.top_k]).explode().nunique())/meta_df.index.nunique()

    score = 0.75*recall + 0.25*ndcg
    rets = {"recall" :recall, 
            "ndcg" :ndcg, 
            "coverage" :coverage, 
            "score" :score}
    return rets

## 예측 DataFrame을 통해 score 평가하기 ## 
def df_to_score(actual_df,predict_df):
    # actual = valid_data_df
    pred = pd.DataFrame()
    query_user_ids = actual_df['profile_id'].unique()
    pred_list=[]
    for user_id in query_user_ids:
        items = predict_df.loc[user_id,:].sort_values(ascending=False).index.values[:25]
        pred_list.append(list(items))

    pred['profile_id'] = query_user_ids
    pred['predicted_list'] = pred_list
    rets = evaluation(actual_df, pred)

    print('score :',rets['score'],'recall :',rets['recall'])

In [35]:
def valid_epoch(cfg, model, data, mode='valid'):
    pred_list = []
    model.eval()
    
    # data -> valid 에 있는 유저만 포함
    query_user_ids = data['profile_id'].unique() # 추론할 모든 user array 집합
    query_user_ids = [real_to_fake_user[real_id] for real_id in query_user_ids]
    full_item_ids = np.array([c for c in range(cfg.n_items)]) # 추론할 모든 fake item array 집합 
    full_item_ids_feat1 = [item_features['genre_mid'][fake_to_real_album[c]] for c in full_item_ids]
    for user_id in query_user_ids:
        with torch.no_grad():
            user_ids = np.full(cfg.n_items, user_id)
            
            user_ids = torch.LongTensor(user_ids).to(cfg.device)
            item_ids = torch.LongTensor(full_item_ids).to(cfg.device)
            
            feat0 = np.full(cfg.n_items, user_features['age'][fake_to_real_user[user_id]])
            feat0 = torch.FloatTensor(feat0).to(cfg.device)
            feat1 = torch.LongTensor(full_item_ids_feat1).to(cfg.device)
            
            eval_output = model.forward(user_ids, item_ids, [feat0, feat1]).detach().cpu().numpy()
            pred_u_score = eval_output.reshape(-1)   
        
        pred_u_idx = np.argsort(pred_u_score)[::-1]
        pred_u = full_item_ids[pred_u_idx]
        pred_list.append([fake_to_real_album[item] for item in list(pred_u[:cfg.top_k])])
        
    pred = pd.DataFrame()
    pred['profile_id'] = [fake_to_real_user[fake_id] for fake_id in query_user_ids]
    pred['predicted_list'] = pred_list
    
    # 모델 성능 확인 
    if mode == 'valid':
        rets = evaluation(data, pred)
        return rets, pred
    return pred

In [36]:
def test_epoch(cfg, model, data, mode='test'):
    pred_list = []
    model.eval()
    actual_pred_matrix=[]
    # data -> valid 에 있는 유저만 포함
    query_user_ids = data['profile_id'].unique() # 추론할 모든 user array 집합
    query_user_ids = [real_to_fake_user[real_id] for real_id in query_user_ids]
    full_item_ids = np.array([c for c in range(cfg.n_items)]) # 추론할 모든 fake item array 집합 
    full_item_ids_feat1 = [item_features['genre_mid'][fake_to_real_album[c]] for c in full_item_ids]
    for user_id in query_user_ids:
        with torch.no_grad():
            user_ids = np.full(cfg.n_items, user_id)
            
            user_ids = torch.LongTensor(user_ids).to(cfg.device)
            item_ids = torch.LongTensor(full_item_ids).to(cfg.device)
            
            feat0 = np.full(cfg.n_items, user_features['age'][fake_to_real_user[user_id]])
            feat0 = torch.FloatTensor(feat0).to(cfg.device)
            feat1 = torch.LongTensor(full_item_ids_feat1).to(cfg.device)
            
            eval_output = model.forward(user_ids, item_ids, [feat0, feat1]).detach().cpu().numpy()
            pred_u_score = eval_output.reshape(-1) 
            
        actual_pred_matrix.append(pred_u_score)
        pred_u_idx = np.argsort(pred_u_score)[::-1]
        pred_u = full_item_ids[pred_u_idx]
        pred_list.append([fake_to_real_album[item] for item in list(pred_u[:cfg.top_k])])
        
    pred = pd.DataFrame()
    pred['profile_id'] = [fake_to_real_user[fake_id] for fake_id in query_user_ids]
    pred['predicted_list'] = pred_list
    
    return pred,actual_pred_matrix

# 모델 학습

### 하이퍼파라미터 설정 & 최적화 기법 설정

In [37]:
# 하이퍼 파라미터 설정 
cfg.batch_size = 256
cfg.emb_dim = 256
cfg.layer_dim = 256
cfg.dropout = 0.05
cfg.epochs = 100
cfg.learning_rate = 0.001
cfg.reg_lambda = 0
cfg.check_epoch = 1

In [45]:
# model 생성 및 optimizer, loss 함수 설정 
model = NeuMF(cfg).to(cfg.device)
optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate, weight_decay=cfg.reg_lambda)

## 회귀 문제로 생각하여 MSELoss 사용 ##
criterion = torch.nn.MSELoss(reduction='sum')

### 학습 진행

In [46]:
total_logs = defaultdict(list)
best_scores  = 0
for epoch in range(cfg.epochs+1):
    cfg.epoch = epoch
    train_results = train_epoch(cfg, model, optimizer, criterion)
    
    # cfg.check_epoch 번의 epoch 마다 성능 확인 
    if epoch % cfg.check_epoch == 0: 
        valid_results, _ = valid_epoch(cfg, model, valid_data_df)

        logs = {
            'Train Loss': train_results['losses'],
            f'Valid Recall@{cfg.top_k}': valid_results['recall'],
            f'Valid NDCG@{cfg.top_k}': valid_results['ndcg'],
            'Valid Coverage': valid_results['coverage'],
            'Valid Score': valid_results['score'],
            }

        # 검증 성능 확인 
        for key, value in logs.items():
            total_logs[key].append(value)

        if epoch == 0:
            print("Epoch", end=",")
            print(",".join(logs.keys()))

        print(f"{epoch:02d}  ", end="")
        print("  ".join([f"{v:0.6f}" for v in logs.values()]))
        
        # 가장 성능이 좋은 가중치 파일을 저장 
        if best_scores <= valid_results['score']: 
            best_scores = valid_results['score']
            torch.save(model.state_dict(), os.path.join(saved_path, 'model(best_scores).pth'))

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch,Train Loss,Valid Recall@25,Valid NDCG@25,Valid Coverage,Valid Score
00  124359.359380  0.134869  0.093091  0.000627  0.124425


  0%|          | 0/33 [00:00<?, ?it/s]

01  1325.547120  0.138453  0.097344  0.000627  0.128176


  0%|          | 0/33 [00:00<?, ?it/s]

...

98  77.072270  0.342948  0.243619  0.283085  0.318115


  0%|          | 0/33 [00:00<?, ?it/s]

99  76.429500  0.343453  0.243191  0.279599  0.318387


  0%|          | 0/33 [00:00<?, ?it/s]

100  76.140500  0.343782  0.244066  0.280326  0.318853


# Valid 데이터에 대한 성능 평가

In [47]:
model.load_state_dict(torch.load(os.path.join(saved_path, 'model(best_scores).pth')))

<All keys matched successfully>

In [48]:
submission_path = os.path.join(data_path, 'sample_submission.csv')
submission = pd.read_csv(submission_path)
_,actual_pred_matrix = test_epoch(cfg, model, submission, mode='test')

In [49]:
actual_pred_matrix_df = pd.DataFrame(actual_pred_matrix,index=ratings_minmax_matrix_df.index,columns=ratings_minmax_matrix_df.columns)
actual_pred_matrix_df

album_id,0,1,2,3,4,5,6,7,8,9,...,25877,25893,25894,25895,25898,25912,25913,25914,25915,25916
profile_id,,,,,,,,,,,,,,,,,,,,,
3,0.016712,0.000882,0.009968,0.001570,0.002289,0.000322,0.000487,0.000019,3.142886e-07,0.000008,...,0.000003,0.000005,0.000005,0.000002,0.000001,0.000003,0.000003,0.000002,0.000002,0.000003
5,0.119002,0.011790,0.007170,0.003831,0.014362,0.003766,0.002836,0.000394,1.464291e-05,0.008282,...,0.000029,0.000025,0.000053,0.000023,0.000026,0.000037,0.000031,0.000047,0.000028,0.000029
7,0.021809,0.008525,0.002797,0.007772,0.003725,0.004332,0.010560,0.002243,6.319171e-05,0.064649,...,0.000106,0.000082,0.000122,0.000080,0.000097,0.000066,0.000059,0.000054,0.000062,0.000060
12,0.004540,0.004498,0.001408,0.002874,0.002822,0.002850,0.000669,0.001750,1.081601e-03,0.001007,...,0.001659,0.001206,0.001582,0.001141,0.000704,0.000795,0.000938,0.000743,0.000983,0.000967
16,0.008157,0.000429,0.000793,0.000351,0.000928,0.001023,0.001206,0.000057,1.184567e-05,0.000275,...,0.000009,0.000005,0.000007,0.000004,0.000011,0.000007,0.000006,0.000005,0.000006,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33022,0.045148,0.031797,0.093549,0.031324,0.021478,0.017225,0.018657,0.000059,7.601582e-06,0.000269,...,0.000007,0.000012,0.000023,0.000008,0.000006,0.000012,0.000012,0.000008,0.000008,0.000011
33023,0.062050,0.003584,0.062567,0.010069,0.003348,0.001933,0.001361,0.000320,3.957174e-05,0.000340,...,0.000060,0.000042,0.000054,0.000026,0.000038,0.000045,0.000037,0.000038,0.000041,0.000037
33026,0.000594,0.000060,0.000040,0.000727,0.000696,0.001168,0.001384,0.000624,2.620764e-03,0.010518,...,0.000617,0.000287,0.000280,0.000287,0.000793,0.000655,0.000486,0.000753,0.000665,0.000423


In [50]:
df_to_score(valid_data_df,actual_pred_matrix_df)

score : 0.31891490250938237 recall : 0.3438036185047568
